In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import numpy.typing as npt
import csv
from tkinter import Tk
from tkinter import filedialog

In [ ]:
def smilesToMolecule(smilesString: str):
    molecule = Chem.AddHs(Chem.MolFromSmiles(smilesString)) 
    AllChem.EmbedMolecule(molecule, AllChem.ETKDGv3())  # Puts the molecule into 3 dimensions

    xyzString = Chem.rdmolfiles.MolToXYZBlock(molecule)
    moleculeXYZ = [i.split() for i in xyzString.split("\n")[2:-1]]

    # List of all atomic symbols in molecule, in order from RDkit generated XYZ file
    atomIdentities = [str(i[0]) for i in moleculeXYZ]
    # Array with all atomic coordinates, without atomic symbols
    atomCoords = np.array([[float(j) for j in i[1:]] for i in moleculeXYZ])

    return atomIdentities, atomCoords

In [ ]:
def substructureMatch(smilesString: str) -> tuple[int, list[int]]:
    """Generate an RDKit molecule and search the structure for protonated atoms."""
    
    molecule = Chem.AddHs(Chem.MolFromSmiles(smilesString))
    params = AllChem.ETKDGv3()
    AllChem.EmbedMolecule(molecule, params)

    chargedAtom = Chem.MolFromSmarts(
        '[#7H+,#7H2+,#7H3+,#8H+,#8H2+]'
    )
    
    protonAtom = Chem.MolFromSmarts(
        '[$([#1][#7H+]),$([#1][#7H2+]),$([#1][#7H3+]),$([#1][#8H+]),$([#1][#8H2+])]'
    )
    
    protonPosition = molecule.GetSubstructMatches(protonAtom)
    parentPosition = molecule.GetSubstructMatches(chargedAtom)

    protonatedAtomIndex = parentPosition[0][0]
    protonPositionIndices = [i[0] for i in protonPosition]

    return protonatedAtomIndex, protonPositionIndices

In [ ]:
class Molecule:
    """Class that contains all molecule data"""

    def __init__(
        self,
        smilesString: str,
        moleculeName: str = None,
        atomIdentities: list[str] = None,
        atomCoords: npt.NDArray = None,
        protonatedAtomIndex: int = None,
        protonPositionIndices: list[int] = None,
        ):
        
        self.smilesString = smilesString
        self.moleculeName = moleculeName if moleculeName is not None else "bean"
        self.atomIdentities = (
            atomIdentities if atomIdentities is not None else smilesToMolecule(self.smilesString)[0]
        )
        self.atomCoords = (
            atomCoords if atomCoords is not None else smilesToMolecule(self.smilesString)[1]
        )
        self.protonatedAtomIndex = (
            protonatedAtomIndex
            if protonatedAtomIndex is not None
            else substructureMatch(self.smilesString)[0]
        )
        self.protonPositionIndices = (
            protonPositionIndices
            if protonPositionIndices is not None
            else substructureMatch(self.smilesString)[1]
        )
        self.translatedAtomCoords = self.atomCoords - self.atomCoords[self.protonatedAtomIndex]


    def getProtonPosition(self, attemptNumber) -> npt.NDArray:
        return self.translatedAtomCoords[int(self.protonPositionIndices[attemptNumber])]
    
    def getProtonatedAtomPosition(self) -> npt.NDArray:
        return self.translatedAtomCoords[self.protonatedAtomIndex]
    
    def getNumAtoms(self) -> int:
        return len(self.atomIdentities)

In [ ]:
# Set commonly used variables

zRotationOffset = np.array(
    [
        [-1, 0, 0], 
        [0, -1, 0], 
        [0,  0, 1]
    ],
    dtype=float
)

# Array of values for the distances between parent atoms
DISTANCE_MULTIPLIERS = [2.7, 2.8, 2.9, 3.0, 3.1, 3.2]

Z_UNIT_VECTOR = np.array([0.0, 0.0, 1.0])

Z_REFLECTOR = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0,-1]
    ],
    dtype=float)

IDENTITY_MATRIX = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ],
    dtype=float)

In [ ]:
def multipleInput() -> tuple[list, str]:
    """Read CSV files with multiple molecules and specify folder containing XYZ files"""

    root = Tk()
    root.wm_attributes('-topmost', 1)
    root.withdraw()

    filename = filedialog.askopenfilename(parent=root,
                                          initialdir = '',
                                          title = 'Select a CSV',
                                          filetypes = (('CSV Files', '*.csv'), ("All files", "*")))

    multipleMoleculeData = []

    with open(filename, newline='') as csvfile:
        data = csv.reader(csvfile)
        for row in data:
            multipleMoleculeData.append(row)

    xyzFilePath = filedialog.askdirectory(initialdir = '',
                                          title = 'Select XYZ File Parent Directory') + '/'

    return multipleMoleculeData[1:], xyzFilePath

In [ ]:
def inputGenerator(
    moleculeName: str,
    stage: str, 
    finalStructure: list[list[str]],
    saveDirectory: str = ''):

    inputSpecs = (
    '!Opt wB97X-D3BJ def2-TZVP LargePrint\n'
    '%geom\n'
    '\tMaxIter 200\n'
    'end\n'
    '%pal\n'
    '\tnprocs 20\n'
    'end\n'
    '* xyz 1 1'
    )

    endLine = '*\n'

    fileName = moleculeName + '-' + stage

    with open(saveDirectory+fileName+'.inp', 'w', newline = '\n') as inputFile:
        np.savetxt(inputFile, finalStructure, fmt = '%s', header = inputSpecs, footer = endLine, comments = '')

In [ ]:
def normalize(vector: npt.NDArray) -> npt.NDArray:
    """Take a vector and return the corresponding unit vector."""

    norm = np.linalg.norm(vector) 

    if norm != 0:
        return vector/norm
    else:
        return vector

In [ ]:
def quaternionBuilder(
    vector: npt.ArrayLike,
    angle: npt.DTypeLike,
    ) -> npt.NDArray:
    """Build a quaternion matrix"""

    quaternion = [
        np.cos(angle/2), 
        vector[0] * np.sin(angle/2), 
        vector[1] * np.sin(angle/2), 
        vector[2] * np.sin(angle/2),
    ]

    # The normalized quaternion vector
    normQuat = normalize(quaternion)

    quaternionMatrix = np.array(
        [
            [
                1 - 2 * (normQuat[2] ** 2 + normQuat[3] ** 2),
                2 * (normQuat[1] * normQuat[2] - normQuat[0] * normQuat[3]),
                2 * (normQuat[1] * normQuat[3] + normQuat[0] * normQuat[2]),
            ],
            [
                2 * (normQuat[1] * normQuat[2] + normQuat[0] * normQuat[3]),
                1 - 2 * (normQuat[1] ** 2 + normQuat[3] ** 2),
                2 * (normQuat[2] * normQuat[3] - normQuat[0] * normQuat[1]),
            ],
            [
                2 * (normQuat[1] * normQuat[3] - normQuat[0] * normQuat[2]),
                2 * (normQuat[2] * normQuat[3] + normQuat[0] * normQuat[1]),
                1 - 2 * (normQuat[1] ** 2 + normQuat[2] ** 2),
            ],
        ]
    )

    return quaternionMatrix

In [ ]:
def vectorAngle(vectorOne: npt.ArrayLike, vectorTwo: npt.ArrayLike) -> np.float64:
    """Calculate the angle (in radians) between two vectors"""

    unitVectorOne = normalize(vectorOne)
    unitVectorTwo = normalize(vectorTwo)

    angle = np.arccos(np.clip(np.dot(unitVectorOne, unitVectorTwo), -1.0, 1.0))

    return angle

In [ ]:
def distanceCalculator(vectorOne: npt.ArrayLike, vectorTwo: npt.ArrayLike) -> np.float64 :
    """Calculate Euclidean distance between two vectors"""

    distance = np.absolute(
        np.sqrt(
            ((float(vectorOne[0]) - float(vectorTwo[0])) ** 2)
            + ((float(vectorOne[1]) - float(vectorTwo[1])) ** 2)
            + ((float(vectorOne[2]) - float(vectorTwo[2])) ** 2)
        )
    )

    return distance

In [ ]:
def xyzFileRead(filePath: str) -> tuple[list[str], npt.NDArray[np.float64]]:
    """Read in XYZ file format and return atomic symbols and coordinates"""

    moleculeXYZ = []

    with open(filePath) as f:
        for line in f:
             # Takes each line in the file minus the last character, which is just the \n
            line = line[:-1].split()
            if line:
                # If the line isn't empty then append to the moleculeXYZ
                moleculeXYZ.append(line) 

    # List of all atomic symbols in molecule, in order from RDkit generated XYZ file
    atomIdentities = [str(i[0]) for i in moleculeXYZ[1:]]

    # Array with all atomic coordinates, without atomic symbols
    atomCoords = np.array([[float(j) for j in i[1:]] for i in moleculeXYZ[1:]])

    return atomIdentities, atomCoords

In [ ]:
def nudgeMatrixGenerator(stage: str) -> tuple[npt.NDArray, npt.NDArray, npt.NDArray]:
    """Creates the nudge matrices and vector for avoiding local minima during optimization.

    `stage` should be either `"R"` (Reactant) or `"P"` (Product) or `"T"` (Transition State)
    """

    # xvDegree (15 Degrees)
    xvDegree = np.pi/12
    # xxDegree (20 Degrees)
    xxDegree = np.pi/9

    nudgeTranslate = np.array([0, 0, 0])
    
    nudgeRotateX = np.empty([3, 3], dtype=float)
    nudgeRotateY = np.empty([3, 3], dtype=float)

    if stage == 'R':
        nudgeRotateX = np.array(
            [
                [1, 0, 0],
                [0, np.cos(xvDegree), -np.sin(xvDegree)],
                [0, np.sin(xvDegree), np.cos(xvDegree)],
            ],
            dtype=float,
        )

        nudgeRotateY = np.array(
            [
                [np.cos(xxDegree), 0, np.sin(xxDegree)],
                [0, 1, 0],
                [-np.sin(xxDegree), 0, np.cos(xxDegree)],
            ],
            dtype=float,
        )
    elif stage == 'P':
        nudgeRotateX = np.array(
            [
                [1, 0, 0],
                [0, np.cos(-xvDegree), -np.sin(-xvDegree)],
                [0, np.sin(-xvDegree), np.cos(-xvDegree)],
            ],
            dtype=float,
        )

        nudgeRotateY = np.array(
            [
                [np.cos(-xxDegree), 0, np.sin(-xxDegree)],
                [0, 1, 0],
                [-np.sin(-xxDegree), 0, np.cos(-xxDegree)],
            ],
            dtype=float,
        )



    return nudgeRotateX, nudgeRotateY, nudgeTranslate

In [ ]:
def checkStructureOneGenerator(
    moleculeAlignmentQuaternion: npt.NDArray, 
    mol: Molecule,
    ) -> list:
    """Align the molecule along Z axis and store its structure"""

    checkStructureOne = []

    for atomPosition in mol.translatedAtomCoords:
        newCoord = np.dot(moleculeAlignmentQuaternion, atomPosition)
        newCoordRounded = [float(round(number,6)) for number in newCoord]
        checkStructureOne.append(newCoordRounded.copy())

    return checkStructureOne

In [ ]:
def checkStructureTwoGenerator(
    stage: str,
    distanceMultiplier: int,
    zRotationOffset: npt.NDArray,
    checkStructureOne: npt.ArrayLike,
    ) -> list:
    """Generate the second structure that the proton will be transferred to"""

    nudgeRotateX, nudgeRotateY, nudgeTranslate = nudgeMatrixGenerator(stage)

    checkStructureTwo = []

    for atomPosition in checkStructureOne:
        if stage == "R":
            newCoord = (
                np.dot(
                    nudgeRotateX,
                    (
                        np.dot(
                            nudgeRotateY,
                            (np.dot(zRotationOffset, (np.dot(Z_REFLECTOR, atomPosition))))
                            + (distanceMultiplier * Z_UNIT_VECTOR),
                        )
                    ),
                )
                + (nudgeTranslate)
            )
        elif stage == "P":
            newCoord = (
                np.dot(
                    nudgeRotateX,
                    (
                        np.dot(
                            nudgeRotateY,
                            (np.dot(zRotationOffset, (np.dot(Z_REFLECTOR, atomPosition))))
                            + (distanceMultiplier * Z_UNIT_VECTOR),
                        )
                    ),
                )
                + ((-1) * nudgeTranslate)
            )
        elif stage == "T":
            newCoord = np.dot(
                zRotationOffset,
                (np.dot(Z_REFLECTOR, atomPosition) + ((distanceMultiplier - 0.1) * Z_UNIT_VECTOR)),
            )
        newCoordRounded = [float(round(number,6)) for number in newCoord]
        checkStructureTwo.append(newCoordRounded.copy())

    return checkStructureTwo

In [ ]:
def structureChecker(
    checkStructureOne: npt.ArrayLike,
    checkStructureTwo: npt.ArrayLike,
    mol: Molecule,
    attemptNumber: int,
    ) -> bool:
    """Check for overlap between the two molecules"""

    n = mol.protonPositionIndices[attemptNumber]
    for i, atomOne in enumerate(checkStructureOne):
        for j, atomTwo in enumerate(checkStructureTwo):
            if (i == n) or (j == n - 1):
                continue
            # Only executes if not proton involved in transport
            if distanceCalculator(atomOne, atomTwo) < 1.5:
                return True
            else:
                continue
    # We checked all atoms, none overlapped
    return False

In [ ]:
def finalStructureGenerator(
    checkStructureOne: npt.ArrayLike, 
    checkStructureTwo: npt.ArrayLike, 
    distanceMultiplier: int,
    stage: str,
    mol: Molecule,
    attemptNumber: int,
    ) -> list[list[str]]:
    """Generate the complete structure"""

    newStructureOne = []
    newStructureTwo = []

    for i in range(len(checkStructureOne)):
        newStructureOne.append([mol.atomIdentities[i]] + checkStructureOne[i])
        newStructureTwo.append([mol.atomIdentities[i]] + checkStructureTwo[i])

    reactantProton = newStructureOne.pop(mol.protonPositionIndices[attemptNumber])
    productProton = newStructureTwo.pop(mol.protonPositionIndices[attemptNumber])
    transitionProton = ['H', 0.0, 0.0, (distanceMultiplier-0.1)/2]

    finalStructure = newStructureOne + newStructureTwo

    if stage == 'R':
        finalStructure.append(reactantProton)
    elif stage == 'P':
        finalStructure.append(productProton)
    elif stage == 'T':
        finalStructure.append(transitionProton)

    return finalStructure

In [ ]:
def overlapHandler(
    checkStructureOne: npt.ArrayLike,
    checkStructureTwo: npt.ArrayLike,
    stage: str,
    distanceMultiplier: float,
    mol: Molecule,
    zRotationOffset: npt.NDArray,
    ) -> tuple[list, list, int]:
    """Fix molecule overlap issues"""

    protonPositionAttempts = len(mol.protonPositionIndices)
    attemptNumber = 0

    while structureChecker(checkStructureOne, checkStructureTwo, mol, attemptNumber) == True and attemptNumber < protonPositionAttempts:

        zRotationOffset = np.array(
            [
                [-1, 0, 0],
                [0, -1, 0],
                [0,  0, 1]
            ],
            dtype=float)

        protonPosition = mol.translatedAtomCoords[int(mol.protonPositionIndices[attemptNumber])]

        moleculeAlignmentAngle = vectorAngle(protonPosition, Z_UNIT_VECTOR)
        moleculeAlignmentVector = normalize(np.cross(protonPosition, Z_UNIT_VECTOR))
        moleculeAlignmentQuaternion = quaternionBuilder(moleculeAlignmentVector, moleculeAlignmentAngle)

        checkStructureOne = checkStructureOneGenerator(moleculeAlignmentQuaternion, mol)
        checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne)
        
        # If the structure works, then return it and exit the function
        if structureChecker(checkStructureOne, checkStructureTwo, mol, attemptNumber) == False:
            print('Structure was fixed with proton position #'+str(attemptNumber))
            return checkStructureOne, checkStructureTwo, attemptNumber

        i = 0
        while structureChecker(checkStructureOne, checkStructureTwo, mol, attemptNumber) == True and i < 8:

            rotationAngle = i * (np.pi/4)

            zRotationOffset = np.array(
                [
                    [np.cos(rotationAngle), -np.sin(rotationAngle), 0],
                    [np.sin(rotationAngle),  np.cos(rotationAngle), 0],
                    [0, 0, 1]
                ],
                dtype=float)
            
            checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne)

            if structureChecker(checkStructureOne, checkStructureTwo, mol, attemptNumber) == False:
                print('Structure was fixed with proton position #'+str(attemptNumber)+' with a rotation of '+str(i*45)+' degrees')
                return checkStructureOne, checkStructureTwo, attemptNumber

            i += 1
        
        attemptNumber += 1
    print('Structure could not be fixed.')
    return checkStructureOne, checkStructureTwo, attemptNumber

In [ ]:
# Structure generation for molecules with pre-existing structures

stage = 'R'

distanceMultiplier = DISTANCE_MULTIPLIERS[0]

multipleMoleculeData, xyzFilePath = multipleInput()

for molecule in multipleMoleculeData:

    moleculeName = molecule[5]
    atomIdentities, atomCoords = xyzFileRead(xyzFilePath+moleculeName+'.xyz')

    molInformation = {
        'protonatedAtomIndex': int(molecule[1]),
        'protonPositionIndices': [int(i) for i in molecule[2:5] if i != 'NA'],
        'moleculeName': moleculeName,
        'atomIdentities': atomIdentities,
        'atomCoords': atomCoords,
    }

    Structure = Molecule(
        molecule[0],
        **molInformation
    )
    
    attemptNumber = 0

    protonPosition = Structure.getProtonPosition(attemptNumber)

    moleculeAlignmentAngle = vectorAngle(protonPosition, Z_UNIT_VECTOR)
    moleculeAlignmentVector = normalize(np.cross(protonPosition, Z_UNIT_VECTOR))
    moleculeAlignmentQuaternion = quaternionBuilder(moleculeAlignmentVector, moleculeAlignmentAngle)

    checkStructureOne = checkStructureOneGenerator(moleculeAlignmentQuaternion, Structure)
    checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne)

    print(moleculeName)

    attemptNumber = 0

    if structureChecker(checkStructureOne, checkStructureTwo, Structure, 0):
        print("PANIC")
        checkStructureOne, checkStructureTwo, attemptNumber = overlapHandler(
            checkStructureOne, 
            checkStructureTwo, 
            stage, 
            distanceMultiplier, 
            Structure, 
            zRotationOffset,
            )

    finalStructure = finalStructureGenerator(
        checkStructureOne,
        checkStructureTwo,
        distanceMultiplier,
        stage,
        Structure,
        attemptNumber
        )

    with open(str(moleculeName)+'-'+stage+'.xyz', 'w', newline='') as FinalFile:
        headerstart = str(Structure.getNumAtoms()*2-1) + '\n'
        np.savetxt(FinalFile, finalStructure, fmt='%s', header=headerstart,comments='')



In [ ]:
# TEST CELL

stage = 'R'

distanceMultiplier = DISTANCE_MULTIPLIERS[0]

multipleMoleculeData = []

with open('MoleculeTest.csv', newline='') as csvfile:
    data = csv.reader(csvfile)
    for row in data:
        multipleMoleculeData.append(row)

atomIdentities, atomCoords = xyzFileRead('TestMolecule.xyz')

molInformation = {
    'protonatedAtomIndex': int(multipleMoleculeData[1][1]),
    'protonPositionIndices': [int(i) for i in multipleMoleculeData[1][2:5] if i != 'NA'],
    'moleculeName': multipleMoleculeData[1][5],
    'atomIdentities': atomIdentities,
    'atomCoords': atomCoords,
}

Structure = Molecule(
    multipleMoleculeData[1][0],
    **molInformation
)

attemptNumber = 0

protonPosition = Structure.getProtonPosition(attemptNumber)

moleculeAlignmentAngle = vectorAngle(protonPosition, Z_UNIT_VECTOR)
moleculeAlignmentVector = normalize(np.cross(protonPosition, Z_UNIT_VECTOR))
moleculeAlignmentQuaternion = quaternionBuilder(moleculeAlignmentVector, moleculeAlignmentAngle)

checkStructureOne = checkStructureOneGenerator(moleculeAlignmentQuaternion, Structure)
checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne)

attemptNumber = 0

if structureChecker(checkStructureOne, checkStructureTwo, Structure, 0):
    print(Structure.moleculeName, 'Warning: Overlap detected, working on fix...')
    checkStructureOne, checkStructureTwo, attemptNumber = overlapHandler(
        checkStructureOne, 
        checkStructureTwo, 
        stage, 
        distanceMultiplier, 
        Structure, 
        zRotationOffset,
        )

finalStructure = finalStructureGenerator(
    checkStructureOne,
    checkStructureTwo,
    distanceMultiplier,
    stage,
    Structure,
    attemptNumber
    )

with open(str(Structure.moleculeName)+'-'+stage+'.xyz', 'w', newline='') as FinalFile:
    headerstart = str(Structure.getNumAtoms()*2-1) + '\n'
    np.savetxt(FinalFile, finalStructure, fmt='%s', header=headerstart,comments='')

In [ ]:
# Molecule generation with only SMILES

stage = 'R'

distanceMultiplier = DISTANCE_MULTIPLIERS[0]

multipleMoleculeData, xyzFilePath = multipleInput()

for molecule in multipleMoleculeData:

    Structure = Molecule(
        molecule[0],
        **{'moleculeName': molecule[1]}
    )
    
    attemptNumber = 0

    protonPosition = Structure.getProtonPosition(attemptNumber)

    moleculeAlignmentAngle = vectorAngle(protonPosition, Z_UNIT_VECTOR)
    moleculeAlignmentVector = normalize(np.cross(protonPosition, Z_UNIT_VECTOR))
    moleculeAlignmentQuaternion = quaternionBuilder(moleculeAlignmentVector, moleculeAlignmentAngle)

    checkStructureOne = checkStructureOneGenerator(moleculeAlignmentQuaternion, Structure)
    checkStructureTwo = checkStructureTwoGenerator(stage, distanceMultiplier, zRotationOffset, checkStructureOne)

    attemptNumber = 0

    if structureChecker(checkStructureOne, checkStructureTwo, Structure, 0):
        print(Structure.moleculeName, 'Overlap detected, working on fix...')
        checkStructureOne, checkStructureTwo, attemptNumber = overlapHandler(
            checkStructureOne,
            checkStructureTwo,
            stage,
            distanceMultiplier,
            Structure,
            zRotationOffset,
            )

    finalStructure = finalStructureGenerator(
        checkStructureOne,
        checkStructureTwo,
        distanceMultiplier,
        stage,
        Structure,
        attemptNumber
        )

    with open(xyzFilePath+str(Structure.moleculeName)+'-'+stage+'.xyz', 'w', newline = '') as finalXYZ:
        headerstart = str(Structure.getNumAtoms()*2-1) + '\n'
        np.savetxt(finalXYZ, finalStructure, fmt = '%s', header = headerstart,comments = '')

    inputGenerator(Structure.moleculeName, stage, finalStructure, xyzFilePath)

with open(xyzFilePath+'run.sh', 'w', newline = '\n') as batch:
    for file in multipleMoleculeData:
        batch.write('$HOME/orca/orca '+file[1]+'-'+stage+'.inp > '+file[1]+'-'+stage+'.out\n')